# Deep learning for salt mapping in 2-D seismic image
This tutorial describes how to map salt in a 2-D seismic image using a deep convolutional neural network. To follow this tutorial, you won't need any special hardware, software, data or experience - just a Google account.

## Setup ##
### Colab ###

1. First, you will need sign in to your Google account. If you're not signed in, you can sign in [here](https://myaccount.google.com/?utm_source=sign_in_no_continue)

2. Next, head on to the [Colab Welcome Page](https://colab.research.google.com/notebooks/welcome.ipynb#recent=true).

3. There, select Github in the top tab and search for https://github.com/thilowrona/seismic_deep_learning/blob/master/Tutorial-1/tutorial-1.ipynb

4. Clicking opens this notebook. This is a Jupyter notebook; an awesome combination of code and documentation allowing us work on, describe and share our projects.

5. Before running anything in here, you need to tell Colab that would like to use a GPU (important to train our models quickly). This is done by clicking on the ‘Runtime’ tab, selecting ‘Change runtime type’, changing hardware accelerator to ´GPU´ and clicking ´save´. Excellent! Colab is now set up!

### Notebooks ###

1. Before using this notebook, you need to install the necessary packages. You can do this by creating a code cell (i.e. clicking ´+ code´ in top left corner), running this command:

In [1]:
 !curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


2. When you run the first cell, you will face a pop-up saying ‘Warning: This notebook was not authored by Google’; you should click on ‘Run Anyway’ to get rid of the warning.

3. Next we want to save our notebook. If you click on ‘File’ and then ‘Save’, you will see a pop-up saying ´CANNOT SAVE CHANGES´. Now, click on ‘SAVE A COPY IN DRIVE’. This opens up a new tab with the same file, but this time located in your Drive. If you want to continue working after saving, use the file in the new tab. Your notebook will be saved in a folder called Colab Notebooks in your Google Drive by default.

4. Last, we would like to load and save files (e.g. our seismic and interpretation). To save files, you need to permit your Colaboratory instance to read and write files to your Google Drive. Luckily, Colab has a UI for permanently mounting your Google Drive. Just click on the files icon on the left sidebar and press the ´Mount Drive´ icon in the top right corner.

After clicking the button you’ll be asked to authorize Colaboratory to access your Google Drive. Once you do, your notebook will automatically mount that drive whenever the runtime is loaded, and the sidebar will show the contents of your drive. Accessing data on your drive is then simply a matter of pointing at the right path. Add the following code snippet at the beginning of every notebook:

In [6]:
root_dir = "drive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

## Load data ##
Now that everything is set up, we can load our data: the seismic image and the labels:

In [2]:
!wget -O data.tif 'https://drive.google.com/file/d/1DnChwuQ03dm8Cosw471LDrG_H6MRfXxb/view?usp=sharing'
!wget -O label.tif 'https://drive.google.com/file/d/1PakAjPRSKjNeTG6jkoCos-jsRRXdwYt7/view?usp=sharing'

--2020-08-19 15:02:48--  https://drive.google.com/file/d/1DnChwuQ03dm8Cosw471LDrG_H6MRfXxb/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 172.217.194.101, 172.217.194.100, 172.217.194.102, ...
Connecting to drive.google.com (drive.google.com)|172.217.194.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data.tif’

data.tif                [ <=>                ]  69.74K  --.-KB/s    in 0.1s    

2020-08-19 15:02:48 (482 KB/s) - ‘data.tif’ saved [71414]

--2020-08-19 15:02:50--  https://drive.google.com/file/d/1PakAjPRSKjNeTG6jkoCos-jsRRXdwYt7/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 74.125.24.113, 74.125.24.100, 74.125.24.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.24.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘label.tif’

label.tif               [ <=>                ]  69.81K  --.-KB/s    i

In [7]:
import matplotlib.pyplot as plt

plt.imread('data.tif')

UnidentifiedImageError: ignored